In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# READ IN DATA

In [3]:
#Loop through csvs and append to df
years = ["2015", "2016", "2017", "2018", "2019", "2020"]
df = pd.DataFrame()
for year in years:
    temp = pd.read_csv(f"../Resources/FY_{year}_Dallas_Animal_Shelter_Data.csv")
    df = df.append(temp, ignore_index = True)

C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Create df with useful columns

clean_df = df[['Animal Id', 'Animal Type', 'Animal Breed', 'Animal Origin', 'Census Tract',
              'Council District', 'Chip Status', 'Intake Date', 'Outcome Date',
              'Intake Condition', 'Outcome Condition', 'Reason', 'Outcome Type', 'Additional Information']]
#clean_df.head()

In [5]:
#Change Date columns from obj to datetime
clean_df['Intake Date'] = pd.to_datetime(clean_df['Intake Date'])
clean_df['Outcome Date'] = pd.to_datetime(clean_df['Outcome Date'])
#clean_df.info()

C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Create a field for time spent in shelter

In [6]:
#Calculate time between intake and outcome, loop through each row
Length_of_Stay = []
for x in range(0, len(clean_df)):
    Length_of_Stay.append(clean_df['Outcome Date'][x] - clean_df['Intake Date'][x])


In [7]:
clean_df['Length of Stay'] = Length_of_Stay
#clean_df.head()

C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Clean Data

In [23]:
#Find potential outliers using IQR.
quartiles = clean_df['Length of Stay'].quantile([.25,.5,.75])

#Use numpy timedelta to convert seconds to days.
lowerq = quartiles[0.25] / np.timedelta64(1, 'D')
upperq = quartiles[0.75] / np.timedelta64(1, 'D')
iqr = upperq-lowerq

print(f"The lower quartile of Length of Stay is: {lowerq}")
print(f"The upper quartile of Length of Stay is: {upperq}")
print(f"The interquartile range of Length of Stay is: {iqr}")
print(f"The the median of Length of Stay is: {quartiles[0.5] / np.timedelta64(1, 'D')} ")

upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of Length of Stay is: 0.0
The upper quartile of Length of Stay is: 8.0
The interquartile range of Length of Stay is: 8.0
The the median of Length of Stay is: 4.0 
Values below -12.0 could be outliers.
Values above 20.0 could be outliers.


## Remove outliers.
#### Because the above value is still a short period of time, we decided to define outliers as stays over 2 months, which is 6.5 * the IQR

In [ ]:
#Remove obvious outliers
clean_df = clean_df.loc[clean_df['Length of Stay'] < "60 days"]

# Clean values to limit number of categories for modeling

In [18]:
#clean intakes

cleaner_df = clean_df[clean_df['Intake Condition'] != "UNHEALTHY UNTREATABLE CONTAGIOUS"]
cleaner_df.loc[cleaner_df['Intake Condition'].str.contains('WNL'), 'Intake Condition'] = 'HEALTHY'
cleaner_df.loc[cleaner_df['Intake Condition'].str.contains('NORMAL'), 'Intake Condition'] = 'HEALTHY'
cleaner_df.loc[cleaner_df['Intake Condition'].str.contains('REHABILITABLE NON-CONTAGIOUS'), 'Intake Condition'] = 'REHABILITABLE NON-CONTAGIOUS'
cleaner_df.loc[cleaner_df['Intake Condition'].str.contains('MANAGEABLE NON-CONTAGIOUS'), 'Intake Condition'] = 'MANAGEABLE NON-CONTAGIOUS'
cleaner_df.loc[cleaner_df['Intake Condition'].str.contains(' CONTAGIOUS'), 'Intake Condition'] = 'CONTAGIOUS'
cleaner_df.loc[cleaner_df['Intake Condition'].str.contains('UNTREATABLE NON-CONTAGIOUS'), 'Intake Condition'] = 'UNTREATABLE NON-CONTAGIOUS'

#Create list and only include values found in list
intakes = ['REHABILITABLE NON-CONTAGIOUS', 'HEALTHY', 'MANAGEABLE NON-CONTAGIOUS',
           'UNTREATABLE NON-CONTAGIOUS', 'CONTAGIOUS']
cleaner_df = cleaner_df[cleaner_df['Intake Condition'].isin(intakes)]
cleaner_df['Intake Condition'].value_counts()

C:\Users\slapp\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


REHABILITABLE NON-CONTAGIOUS    125162
HEALTHY                          32995
UNTREATABLE NON-CONTAGIOUS       21364
MANAGEABLE NON-CONTAGIOUS        17087
CONTAGIOUS                        2487
Name: Intake Condition, dtype: int64

In [19]:
#Clean outcome types

cleaner_df.loc[df['Outcome Type'].str.contains('FOSTER'), 'Outcome Type'] = 'TRANSFER'
outcomes = ['ADOPTION', 'RETURNED TO OWNER', 'TRANSFER','EUTHANIZED']
cleaner_df = cleaner_df[cleaner_df['Outcome Type'].isin(outcomes)]
cleaner_df['Outcome Type'].value_counts()

ADOPTION             64754
TRANSFER             45006
EUTHANIZED           42255
RETURNED TO OWNER    32398
Name: Outcome Type, dtype: int64

In [20]:
#Clean chip status
cleaner_df.loc[cleaner_df['Chip Status'].str.contains('SCAN NO CHIP'), 'Chip Status'] = 'NO CHIP'
cleaner_df.loc[cleaner_df['Chip Status'].str.contains('SCAN CHIP'), 'Chip Status'] = 'CHIP'
cleaner_df.loc[cleaner_df['Chip Status'].str.contains('UN'), 'Chip Status'] = 'UNABLE TO SCAN'
cleaner_df['Chip Status'].value_counts()

NO CHIP           132255
CHIP               38698
UNABLE TO SCAN     13460
Name: Chip Status, dtype: int64

In [22]:
#clean origins
cleaner_df = cleaner_df.dropna(subset=['Animal Origin'])
cleaner_df.loc[cleaner_df['Animal Origin'].str.contains('SWEEP'), 'Animal Origin'] = 'FIELD'
cleaner_df.loc[cleaner_df['Animal Origin'].str.contains('OPS'), 'Animal Origin'] = 'FIELD'

cleaner_df.loc[cleaner_df['Animal Origin'].str.contains('CARE'), 'Animal Origin'] = 'OVER THE COUNTER'
cleaner_df.loc[cleaner_df['Animal Origin'].str.contains('NIGHT DROP'), 'Animal Origin'] = 'OVER THE COUNTER'

origins = ['FIELD', 'OVER THE COUNTER']
cleaner_df = cleaner_df[cleaner_df['Animal Origin'].isin(origins)]
cleaner_df['Animal Origin'].value_counts()

FIELD               94616
OVER THE COUNTER    89722
Name: Animal Origin, dtype: int64

# Split into cat and dog dfs for more cleaning before modeling


In [23]:
dog_df = cleaner_df.loc[cleaner_df['Animal Type'] == "DOG"]
cat_df = cleaner_df.loc[cleaner_df['Animal Type'] == "CAT"]
len(dog_df)

136303

In [24]:
dog_df = dog_df[['Animal Type', 'Animal Id', 'Animal Breed',
       'Animal Origin', 'Census Tract', 'Council District', 'Chip Status',
       'Intake Date', 'Outcome Date', 'Intake Condition', 'Outcome Condition',
       'Outcome Type', 'Length of Stay']]

In [25]:
#Clean dog breeds
dog_df.loc[dog_df['Animal Breed'].str.contains('PIT'), 'Animal Breed'] = 'PIT BULL'
dog_df.loc[dog_df['Animal Breed'].str.contains('STAFF'), 'Animal Breed'] = 'PIT BULL'

dog_df.loc[dog_df['Animal Breed'].str.contains('TERRIER'), 'Animal Breed'] = 'TERRIER'
dog_df.loc[dog_df['Animal Breed'].str.contains('TERR'), 'Animal Breed'] = 'TERRIER'
dog_df.loc[dog_df['Animal Breed'].str.contains('TER'), 'Animal Breed'] = 'TERRIER'
dog_df.loc[dog_df['Animal Breed'].str.contains('WEST HIGH'), 'Animal Breed'] = 'TERRIER'
dog_df.loc[dog_df['Animal Breed'].str.contains('SCHNAUZER'), 'Animal Breed'] = 'TERRIER'


dog_df.loc[dog_df['Animal Breed'].str.contains('RETR'), 'Animal Breed'] = 'RETRIEVER'
dog_df.loc[dog_df['Animal Breed'].str.contains('SHEPHERD'), 'Animal Breed'] = 'SHEPHERD'
dog_df.loc[dog_df['Animal Breed'].str.contains('CORGI'), 'Animal Breed'] = 'CORGI'

dog_df.loc[dog_df['Animal Breed'].str.contains('HOUND'), 'Animal Breed'] = 'HOUND'
dog_df.loc[dog_df['Animal Breed'].str.contains('BEAGLE'), 'Animal Breed'] = 'HOUND'
dog_df.loc[dog_df['Animal Breed'].str.contains('BLACK MOUTH'), 'Animal Breed'] = 'HOUND'
dog_df.loc[dog_df['Animal Breed'].str.contains('DACHSHUND'), 'Animal Breed'] = 'HOUND'
dog_df.loc[dog_df['Animal Breed'].str.contains('RIDGEBACK'), 'Animal Breed'] = 'HOUND'

dog_df.loc[dog_df['Animal Breed'].str.contains('CHIHUAHUA'), 'Animal Breed'] = 'CHIHUAHUA'

In [26]:
#Select top breeds, other, and Corgis and Pugs for suck up points
dog_breeds = ['PIT BULL', 'CHIHUAHUA', 'SHEPHERD', 'RETRIEVER', 'TERRIER', 'HOUND', 'CORGI', 'PUG']
dog_df.loc[~dog_df["Animal Breed"].isin(dog_breeds), "Animal Breed"] = "OTHER"

In [27]:
dog_df['Animal Breed'].value_counts()

PIT BULL     34023
OTHER        29127
CHIHUAHUA    19659
SHEPHERD     18717
RETRIEVER    18337
TERRIER      10874
HOUND         4648
CORGI          509
PUG            409
Name: Animal Breed, dtype: int64

In [28]:
dog_df.head(1)

,Animal Type,Animal Id,Animal Breed,Animal Origin,Census Tract,Council District,Chip Status,Intake Date,Outcome Date,Intake Condition,Outcome Condition,Outcome Type,Length of Stay
1,DOG,A0008962,RETRIEVER,FIELD,75218,18,NO CHIP,2015-09-24,2015-10-04,REHABILITABLE NON-CONTAGIOUS,TREATABLE MANAGEABLE NON-CONTAGIOUS,EUTHANIZED,10 days


In [29]:
cat_df = cat_df[['Animal Type', 'Animal Id', 'Animal Breed',
       'Animal Origin', 'Census Tract', 'Council District', 'Chip Status',
       'Intake Date', 'Outcome Date', 'Intake Condition', 'Outcome Condition',
       'Outcome Type', 'Length of Stay']]

In [30]:
#Clean cat breeds
cat_df.loc[cat_df['Animal Breed'].str.contains('SIAMESE'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('RUSSIAN BLUE'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('BOMBAY'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('SAVANNAH'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('BURMESE'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('HAVANA BROWN'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('COLORPOINT'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('KORAT'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('JAPAN BOBTAIL'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('TONKINESE'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('SPHYNX'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('TONKINESE'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('CORNISH REX'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('OCICAT'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('TONKINESE'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('EGYPTIAN MAU'), 'Animal Breed'] = 'DOMESTIC SH'
cat_df.loc[cat_df['Animal Breed'].str.contains('SH'), 'Animal Breed'] = 'DOMESTIC SH'


cat_df.loc[cat_df['Animal Breed'].str.contains('ABYSSINIAN'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('CHARTREUX'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('BENGAL'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('RAGDOLL'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('BALINESE'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('MANX'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('SCOTTISH FOLD'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('SNOWSHOE'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('AMER WIREHAIR'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('SOMALI'), 'Animal Breed'] = 'DOMESTIC MH'
cat_df.loc[cat_df['Animal Breed'].str.contains('MH'), 'Animal Breed'] = 'DOMESTIC MH'

cat_df.loc[cat_df['Animal Breed'].str.contains('HIMALAYAN'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('MAINE COON'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('PERSIAN'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('NORWGN FOREST'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('ANGORA'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('TURKISH VAN'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('BIRMAN'), 'Animal Breed'] = 'DOMESTIC LH'
cat_df.loc[cat_df['Animal Breed'].str.contains('LH'), 'Animal Breed'] = 'DOMESTIC LH'



In [31]:
cat_df['Animal Breed'].value_counts()

DOMESTIC SH    39945
DOMESTIC MH     3000
DOMESTIC LH      914
Name: Animal Breed, dtype: int64

In [32]:
cat_df.head(1)

,Animal Type,Animal Id,Animal Breed,Animal Origin,Census Tract,Council District,Chip Status,Intake Date,Outcome Date,Intake Condition,Outcome Condition,Outcome Type,Length of Stay
0,CAT,A0000575,DOMESTIC SH,OVER THE COUNTER,W,W,NO CHIP,2014-10-02,2014-10-12,REHABILITABLE NON-CONTAGIOUS,TREATABLE REHABILITABLE NON-CONTAGIOUS,ADOPTION,10 days


# Write to CSV

In [ ]:
cleaner_df.to_csv('../Resources/combined_shelter_data.csv')
cat_df.to_csv('../Resources/cat.csv')
dog_df.to_csv('../Resources/dog.csv')